<a href="https://colab.research.google.com/github/KendyHuynh/E-commer-web/blob/main/Project_BigData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [ ]:
# Cài đặt PySpark
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=55604e4a8219848c67c48421a49677238741fc7de3c060160db756c6cdbf7919
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# Khởi tạo SparkSession và đọc dữ liệu từ tập tin log:

Sử dụng SparkSession để khởi tạo một phiên làm việc Spark.

Sử dụng phương thức read.text() để đọc dữ liệu từ tập tin log vào một DataFrame.

In [ ]:
from pyspark.sql import SparkSession

# Khởi tạo SparkSession
spark = SparkSession.builder \
    .appName("readdatalog") \
    .getOrCreate()

# Đường dẫn đến tập tin log
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Đọc dữ liệu log từ tập tin
log_data = spark.read.text(log_file_path)

# Hiển thị schema của DataFrame
log_data.printSchema()

# Hiển thị 20 dòng đầu tiên của DataFrame
log_data.show(20, truncate=False)

# Đóng SparkSession
spark.stop()



root
 |-- value: string (nullable = true)

+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|value                                                                                                                                                                                                                                                                                                                                                                                                                                 

**Nhìn vào dữ liệu log đọc được ta thấy rằng có các thông tin như sau:**

1. IP_Address (31.56.96.51)

2. TimeStamp ([22/Jan/2019...])

3. Request_HTTP ("GET /image/60844/productModel/...")

4. Status_Code (200)

5. Response_Size (5667)

6. Referrer ("https://www.zanbil.ir/...")

7. User_Agent ("Mozilla/5.0 (Linux; Android 6.0...")

8. Additional_Info ("-")

#Phân tích log và tạo DataFrame mới:

Sử dụng biểu thức chính quy để trích xuất thông tin từ dòng log, bao gồm địa chỉ IP, thời gian, yêu cầu, mã trạng thái, kích thước phản hồi, referrer, user agent và thông tin bổ sung.

Sử dụng regexp_extract() để áp dụng biểu thức chính quy và tạo các cột mới trong DataFrame.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import regexp_extract

# Khởi tạo SparkSessi on
spark = SparkSession.builder \
    .appName("LogAnalysis") \
    .getOrCreate()

# Đường dẫn đến file logs
log_file_path = "/content/drive/MyDrive/Colab Notebooks/Dữ liệu lớn/Đồ án/Datalog/access.log"

# Biểu thức regex để trích xuất thông tin từ dòng log
log_regex = r'^(\S+) - - \[(.*?)\] "(.*?)" (\d+) (\d+|-) "(.*?)" "(.*?)" "(.*?)"$'

# Đọc dữ liệu từ file logs vào DataFrame và phân tích các trường thông tin
parsed_logs_df = spark.read.text(log_file_path).select(
    regexp_extract('value', log_regex, 1).alias('ip_address'),
    regexp_extract('value', log_regex, 2).alias('timestamp'),
    regexp_extract('value', log_regex, 3).alias('request'),
    regexp_extract('value', log_regex, 4).alias('status_code'),
    regexp_extract('value', log_regex, 5).alias('response_size'),
    regexp_extract('value', log_regex, 6).alias('referrer'),
    regexp_extract('value', log_regex, 7).alias('user_agent'),
    regexp_extract('value', log_regex, 8).alias('additional_info')
)

# Hiển thị DataFrame sau khi đã phân tích các trường thông tin
parsed_logs_df.show(50,truncate=False)

+-------------+--------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------+-------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+---------------+
|ip_address   |timestamp                 |request                                                                                                                                                                                                      |status_code|response_size|referrer              

In [ ]:
# Hiển thị số lượng dòng và cột trong DataFrame
print("Số lượng dòng và cột trong DataFrame:")
print((parsed_logs_df.count(), len(parsed_logs_df.columns)))

Số lượng dòng và cột trong DataFrame:


NameError: name 'parsed_logs_df' is not defined

In [ ]:
# Mô tả dữ liệu
print("Mô tả dữ liệu:")
parsed_logs_df.describe().show()

Mô tả dữ liệu:


NameError: name 'parsed_logs_df' is not defined

In [ ]:
# Phân phối dữ liệu theo cột status_code
print("Phân phối dữ liệu theo cột status_code:")
parsed_logs_df.groupBy('status_code').count().show()

In [ ]:
# Tần suất của mỗi giá trị trong cột status_code
print("Tần suất của mỗi giá trị trong cột status_code:")
parsed_logs_df.groupBy('status_code').count().orderBy('count', ascending=False).show()

In [ ]:
# Vẽ biểu đồ trực quan hóa
import matplotlib.pyplot as plt

# Biểu đồ phân phối dữ liệu theo cột status_code
status_code_counts = parsed_logs_df.groupBy('status_code').count().toPandas()
status_code_counts.plot(kind='bar', x='status_code', y='count', title='Phân phối dữ liệu theo status_code')
plt.xlabel('Status Code')
plt.ylabel('Count')
plt.show()